# Sports14 Text/Image Feature Extraction

In [2]:

import os
import numpy as np
import pandas as pd

In [3]:
os.chdir('/nfs/user/MMRec/Amazon_Elec_Dataset')
os.getcwd()

'/nfs/user/MMRec/Amazon_Elec_Dataset'

## Load text data

In [4]:
i_id, desc_str = 'itemID', 'description'

file_path = './'
file_name = 'meta-elec.csv'

meta_file = os.path.join(file_path, file_name)

df = pd.read_csv(meta_file)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (63001, 10)


,itemID,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[['Electronics', 'GPS & Navigation', 'Vehicle ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,1,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[['Electronics', 'Computers & Accessories', 'T...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",NaN
2,2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,19.65,NaN,"{'also_bought': ['B00AAKLIIS', 'B00A668GUO', '...",Barnes &amp; Noble


In [5]:

# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

(1181, 10)
(8979, 10)
(385, 10)
(385, 10)
(0, 10)


In [6]:

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")


In [7]:
# item_id
item_id = df[i_id].tolist()
print(len(item_id))

63001


In [8]:
sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

# sentences[:10]
print(type(sentences))
print(len(sentences))
print(sentences[:10])

<class 'list'>
63001
['Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS   Electronics GPS & Navigation Vehicle GPS Trucking GPS Like its award-winning predecessor, the IntelliRoute TND 500, the Rand McNally IntelliRoute TND 700 GPS device gives truckers much more than just maps and navigation. It offers detailed information about truck stops, rest areas, weigh stations and other points of interest for truckers, as well as tools to help drivers estimate the profitability of a route. And with its seven-inch screen and high-definition display, the IntelliRoute TND 700 redefines readability and usability. Other exclusive features include an oversized stylus, larger buttons, a louder speaker, and tools like a calendar and notepad to help professional drivers complete their routes with greater ease and efficiency.IntelliRoute TND 700At a glance:35-percent more truck-routing information than other GPS devicesSeven-inch, high-definition screenLarge buttons and powerful speaker opti

In [11]:
#make a dict item_id & text
dict_iid_text = dict(zip(item_id,sentences))
print(dict_iid_text[0])

Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS   Electronics GPS & Navigation Vehicle GPS Trucking GPS Like its award-winning predecessor, the IntelliRoute TND 500, the Rand McNally IntelliRoute TND 700 GPS device gives truckers much more than just maps and navigation. It offers detailed information about truck stops, rest areas, weigh stations and other points of interest for truckers, as well as tools to help drivers estimate the profitability of a route. And with its seven-inch screen and high-definition display, the IntelliRoute TND 700 redefines readability and usability. Other exclusive features include an oversized stylus, larger buttons, a louder speaker, and tools like a calendar and notepad to help professional drivers complete their routes with greater ease and efficiency.IntelliRoute TND 700At a glance:35-percent more truck-routing information than other GPS devicesSeven-inch, high-definition screenLarge buttons and powerful speaker optimized for truck cabSpok

In [23]:
#print for llama
print(dict_iid_text[24305])

Gigabyte GA-P55M - UD2 Ultra Durable 3 Motherboard   Electronics Computers & Accessories Computer Components Motherboards Gigabyte GA-P55M-UD2 Intel Core i7 / i5/ LGA1156/ 4DDR3-2000/ CrossFire/ GbE/ Raid/ 1394 Micro ATX Motherboard


import json
jsons = json.dumps(dict_iid_text)
with open("raw_text.json",'w') as f:
    f.write(jsons)
print('raw text json done')